# Grover Algorithm - 2 and 3 qubits
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$

In [1]:
from pyquil.quil import Program
from pyquil.gates import *
from pyquil import get_qc
import numpy as np
import pandas as pd

In [2]:
def create_grover_circuit(oracle, core_phase, qubit_list):
    n_qubit = len(qubit_list)
    circuit = Program()
    for i in range(n_qubit):
        circuit += H(qubit_list[i])
    for gate in oracle:
        circuit += gate    
    # Start Inversion About the Mean
    for i in range(n_qubit):
        circuit += H(qubit_list[i])   
    for gate in core_phase:
        circuit += gate
    for i in range(n_qubit):
        circuit += H(qubit_list[i])
    # End Inversion About the Mean        
    ro = circuit.declare('ro', 'BIT', n_qubit)
    for i in range(n_qubit):
        circuit += MEASURE(qubit_list[i], ro[i])
    return circuit

In [3]:
def run_circuit(circuit, qubit_list, qvm):
    n = len(qubit_list)
    exp_array = np.array([])
    for i in range(n):
        exp_array = np.append(exp_array,2**i)
    shots = 10**4
    circuit.wrap_in_numshots_loop(shots)
    #qc = get_qc('4q-qvm')  # You can make any 'nq-qvm' this way for any reasonable 'n'
    qc = get_qc("Aspen-M-1", as_qvm=qvm, execution_timeout=60)
    executable = qc.compile(circuit)
    result = qc.run(executable)
    bitstrings = result.readout_data.get('ro')[:,:n]
    results = bitstrings.dot(exp_array)
    unique_elements, counts_elements = np.unique(results, return_counts=True)
    print(pd.DataFrame(counts_elements, index = unique_elements,columns=["N"]))

## 2 Qubits 

Best qubit in term of fidelity on the Aspen-M-1 Rigetti Processor:

\begin{align}
f(x_0,x_1) = x_0\cdot x_1\;\;\; \Rightarrow \;\;\; U_f\ket{x}=(-1)^{x_0\cdot x_1}\ket{x}
\end{align}

In [4]:
#Aspen-M-1-----------
q_0 = 17
q_1 = 114

qubit_list = [q_0,q_1]
oracle = [CZ(q_0,q_1)]
core_phase = [Z(q_0),Z(q_1),CZ(q_0,q_1)]

In [5]:
circuit = create_grover_circuit(oracle, core_phase, qubit_list)

In [6]:
run_circuit(circuit, qubit_list, qvm = True)

         N
3.0  10000


In [7]:
run_circuit(circuit, qubit_list, qvm = False)

        N
0.0    28
1.0  1657
2.0   131
3.0  8184


Results after 10,000 run. The states are:

|Index|Quantum State|
|:---------:|:---------:|
|0|\|00>|
|1|\|01>|
|2|\|10>|
|3|\|11>|

## 3 Qubits 

Best qubit in term of fidelity on the Aspen-M-1 Rigetti Processor:

In [8]:
#Aspen-M-1-----------
q_0 = 35
q_1 = 21
q_2 = 36

qubit_list = [q_0,q_1,q_2]
oracle = [CZ(q_0,q_2),CZ(q_1,q_2)]
core_phase = [X(q_0),X(q_1),X(q_2),Z(q_2).controlled(q_0).controlled(q_1),X(q_0),X(q_1),X(q_2)]

In [9]:
circuit = create_grover_circuit(oracle, core_phase, qubit_list)

In [10]:
run_circuit(circuit, qubit_list, qvm = True)

        N
5.0  5030
6.0  4970


In [11]:
run_circuit(circuit, qubit_list, qvm = False)

        N
0.0   184
1.0   324
2.0   661
3.0   332
4.0   534
5.0  3293
6.0  4233
7.0   439


Results after 10,000 run. The states are:

|Index|Quantum State|
|:---------:|:---------:|
|0|\|000>|
|1|\|001>|
|2|\|010>|
|3|\|011>|
|4|\|100>|
|5|\|101>|
|6|\|110>|
|7|\|111>|
